# Extract for DATA3998

## Notes
This notebook is intended to transparently document the data extract preparation for the
capstone projects. You do not need to rerun this notebook. Please use the `*.csv` files that
were provided with this notebook.

## Usage
Carefully review each section in this notebook. Metadata descriptions and citations to the
original source are provided for each table. In addition to the tables listed below three
additional reference tables are provided.

## Country
The `Country.csv` file maps two character country codes to full names. The codes were parsed
from metadata in the
[Provider Enumeration System](https://www.nber.org/research/data/national-plan-and-provider-enumeration-system-nppes)
available from the [National Bureau of Economic Research](https://www.nber.org/). This file
contains less than 1000 records. The unique `Code` can be linked to the `Country` in the
`Provider` table.

### Record Layout
|Field |Type     |Description                                                                              |
|------|---------|-----------------------------------------------------------------------------------------|
|`Code`|`CHAR(2)`|Country unique identifier. Leading characters are meaningful, including zeros and spaces.|
|`Name`|`VARCHAR`|Full name of the country.                                                                |

### Quality Control

Regular Expression to find invalid records.

```Re
^(?!"[A-Z]{2}","[^"]*"$)
```

## State
The `State.csv` file maps two character state codes to full names. The codes were parsed
from metadata in the
[Provider Enumeration System](https://www.nber.org/research/data/national-plan-and-provider-enumeration-system-nppes)
available from the [National Bureau of Economic Research](https://www.nber.org/). This file
contains less than 100 records. The unique `Code` can be linked to the `State` in the
`Provider` table.

### Record Layout
|Field       |Type     |Description                                                                            |
|------------|---------|---------------------------------------------------------------------------------------|
|`Code`      |`CHAR(2)`|State unique identifier. Leading characters are meaningful, including zeros and spaces.|
|`Name`      |`VARCHAR`|Full name of the state.                                                                |
|`Governance`|`VARCHAR`|Jurisdictional governance. Values are `STATE` or `TERRITORY`.                          |

### Quality Control

Regular Expression to find invalid records.

```Re
^(?!"[A-Z]{2}","[^"]*","(STATE|TERRITORY)"$)
```

## Taxonomy
The `Taxonomy.csv` file maps 10 character clinical specialty taxonomy codes to full
descriptions. The file is provided as is from the
[National Uniform Claim Committee](https://nucc.org/). This file contains less than 1000
records and was processed in less than one second. The unique `Code` can be linked to the
`Code` in the `Specialty` table. 

### Record Layout
|Field           |Type      |Description                                                                                         |
|----------------|----------|----------------------------------------------------------------------------------------------------|
|`Code`          |`CHAR(10)`|Clinical Specialty unique identifier. Leading characters are meaningful, including zeros and spaces.|
|`Grouping`      |`VARCHAR` |Highest level domain of clinical practice.                                                          |
|`Classification`|`VARCHAR` |Discipline or field of clinical practice within the high level domain.                              |
|`Specialization`|`VARCHAR` |Specialization within the discipline of clinical practice.                                          |
|`Name`          |`VARCHAR` |Amalgamation of the domain, discipline, and specialization.                                         |
|`Individual`    |`BOOLEAN` |Indicates whether the taxonomy category applies to individual practice or an organization.          |

### Quality Control

Regular Expression to find invalid records.

```Re
^(?!"[A-Z0-9]{10}","[^"]*","[^"]*","[^"]*","[^"]*","(True|False)"$)
```

### Files
* Source `Data\nucc_taxonomy_240.csv`
* Target `Data\Clean\Taxonomy.csv`

In [4]:
# Usual suspects
import pandas as pd
from csv import QUOTE_ALL

# Read
taxonomysource = pd.read_csv("Data\\nucc_taxonomy_240.csv", dtype = str)

# Filter columns
taxonomysource.replace("\"+", " ", regex = True)
taxonomysource["Individual"] = (taxonomysource["Section"] == "Individual")
taxonomysource.rename(
    columns = { "Display Name": "Name" },
    inplace = True
)
taxonomytarget = taxonomysource[
    [
        "Code",
        "Grouping",
        "Classification",
        "Specialization",
        "Name",
        "Individual"
    ]
]

# Write quoting all the string fields
taxonomytarget.to_csv(
    "Data\Clean\Taxonomy.csv",
    index = False,
    quoting = QUOTE_ALL
)

# Display complete
print(f"Records: {len(taxonomytarget.index)}", end = "\r")

## Clinical Specialties

The clinical specialties of the clinical providers were obtained from the
[Provider Enumeration System](https://www.nber.org/research/data/national-plan-and-provider-enumeration-system-nppes)
available from the [National Bureau of Economic Research](https://www.nber.org/). The file
was filtered down to just the primary specialty of each clinical provider, extraneous fields
were removed, and the columns were renamed to replace jargon and abbreviations with a
concise human readable label. This file contains approximately 7000000 records and was
processed within one minute using Python. The unique `Provider` field can be linked to the
`Provider` field in the `Provider` table, and the `SourceProvider` and `TargetProvider` 
fields in `Followup` table.


### Record Layout
|Field     |Type      |Description                                                                                                               |
|----------|----------|--------------------------------------------------------------------------------------------------------------------------|
|`Provider`|`CHAR(10)`|Clinical provider's unique identifier. Leading characters are meaningful, including zeros and spaces.                     |
|`Code`    |`CHAR(10)`|Taxonomy code of the clinical provider's primary specialty. Leading characters are meaningful, including zeros and spaces.|

### Quality Control

Regular Expression to find invalid records.

```Re
^(?!"[0-9]{10}","[A-Z0-9]{10}"$)
```

### Files
* Source `Data\ptaxcode20235.csv`
* Target `Data\Clear\Specialty.csv`

Step through in blocks of 10000 appending to the file.

In [6]:
# Usual suspects
import pandas as pd
from csv import QUOTE_ALL

# Clean up, rename, remove columns
def cleanspecialty(source):
    source.replace("\"+", " ", regex = True)
    source.rename(
        columns = {
            "npi": "Provider",
            "ptaxcode": "Code"
        },
        inplace = True
    )
    target = source[source["pprimtax"] == "Y"][
        [
            "Provider",
            "Code"
        ]
    ]
    return target


# Batch read
chunksize = 10**4
records = 0
with pd.read_csv("Data\ptaxcode20235.csv", dtype = str, chunksize = chunksize) as specialtyreader:

    # First chunk
    specialtytarget = cleanspecialty(next(specialtyreader))

    # Write with header
    specialtytarget.to_csv(
        "Data\Clean\Specialty.csv", 
        quoting = QUOTE_ALL,
        index = False
    )

    # Progress
    records += len(specialtytarget.index)
    print(f"Records: {records}", end = "\r")

    # All other chunks
    for specialtysource in specialtyreader:
        specialtytarget = cleanspecialty(specialtysource)

        # Append without header
        specialtytarget.to_csv(
            "Data\Clean\Specialty.csv",
            mode = "a",
            quoting = QUOTE_ALL,
            header = False,
            index = False
        )

        # Progress
        records += len(specialtytarget.index)
        print(f"Records: {records}", end = "\r")

## Providers
The addresses and practice types of clinical providers were obtained from the
[Provider Enumeration System](https://www.nber.org/research/data/national-plan-and-provider-enumeration-system-nppes)
available from the [National Bureau of Economic Research](https://www.nber.org/). Extraneous
fields were removed, the address fields were coalesced to minimize missing values, the
practice type was coerced into a Boolean indicator, and the columns were renamed to replace
jargon and abbreviations with a concise human readable label. This file contains
approximately 7000000 records and was processed within 10 minutes in Python. The unique
`Provider` field can be linked to the `Provider` field in the `Specialty` table, and the
`SourceProvider` and `TargetProvider` fields in `Followup` table.

### Record Layout
|Field       |Type      |Description                                                                                            |
|------------|----------|-------------------------------------------------------------------------------------------------------|
|`Provider`  |`CHAR(10)`|Clinical provider's unique identifier. Leading characters are meaningful, including zeros and spaces.  |
|`Individual`|`BOOLEAN` |Indicator of individual practice. `True` indicates individual practice, `False` indicates organization.|
|`Zip`       |`VARCHAR` |Postal Zip+4 code of the provider's practice location.                                                 |
|`City`      |`VARCHAR` |City of the provider's practice location.                                                              |
|`State`     |`CHAR(2)` |State of the provider's practice location. Maps to the `Code` in the `State.csv` file.                 |
|`Country`   |`CHAR(2)` |Country of the provider's practice location. Maps to the `Code` in the `Country.csv` file.             |

### Quality Control

Regular Expression to find invalid records.

```Re
^(?!"[0-9]{10}","(True|False)","[^"]*","[^"]*","([A-Z][A-Z.])?","([A-Z]{2})?"$)
```

### Files
* Source `Data\core20235.csv`
* target `Data\Clean\Provider.csv`

Step through in blocks of 10000 appending to the file.

In [7]:
# Usual suspects
import pandas as pd
from csv import QUOTE_ALL

# Clean up, rename, remove columns
def cleanproviders(source):
    source.replace("\"+", " ", regex = True)
    source.rename(columns = { "npi": "Provider" }, inplace = True)
    source["Individual"] = (source["entity"] == "1")
    source["Zip"] = source["ploczip"].where(pd.notnull, source["pmailzip"])
    source["City"] = source["ploccityname"].where(pd.notnull, source["pmailcityname"])
    source["State"] = source["plocstatename"].where(pd.notnull, source["pmailstatename"])
    source["Country"] = source["ploccountry"].where(pd.notnull, source["pmailcountry"])
    source["State"] = source["State"].str.replace(
        "((?<=[A-Z])[A-Z]+(?=[A-Z]))|([^A-Z]+)|((?<=[A-Z])[A-Z](?=[^A-Z]+[A-Z]))",
        "",
        regex = True
    ).str.replace(
        "(?<=^[A-Z])$", ".", regex = True
    ).str.slice(start = 0, stop = 2)
    source["Zip"] = source["Zip"].str.replace("(?<=^[0-9]{5})$", "0000", regex = True)
    target = source[
        [
            "Provider",
            "Individual",
            "Zip",
            "City",
            "State",
            "Country"
        ]
    ]
    return target

# Batch read
chunksize = 10**4
records = 0
with pd.read_csv("Data\core20235.csv", dtype = str, chunksize = chunksize) as providerreader:

    # First chunk
    providertarget = cleanproviders(next(providerreader))

    # Write with header
    providertarget.to_csv(
        "Data\Clean\Provider.csv", 
        quoting = QUOTE_ALL,
        index = False
    )

    # Progress
    records += len(providertarget.index)
    print(f"Records: {records}", end = "\r")

    # All other chunks
    for providersource in providerreader:
        providertarget = cleanproviders(providersource)

        # Append without header
        providertarget.to_csv(
            "Data\Clean\Provider.csv",
            mode = "a",
            quoting = QUOTE_ALL,
            header = False,
            index = False
        )

        # Progress
        records += len(providertarget.index)
        print(f"Records: {records}", end = "\r")

## Followups
The records of followup encounters between pairs of clinical providers were obtained from
the
[Physician Shared Patients](https://www.nber.org/research/data/physician-shared-patient-patterns-data)
available from the [National Bureau of Economic Research](https://www.nber.org/). Extraneous
fields were removed, and the columns were renamed to replace jargon and abbreviations with a
concise human readable label.

Each file contains transactional data derived from a single year of Medicaid/Medicare use.
A single record in the file corresponds to a unique pair of clinicians, and measures the
total volume of followups with the followup clinical provider that occurred within a month
of a clinical encounter with the initial clinical provider. In the course of a calendar year
a client may have had multiple followup pairs of encounters. There are approximately
55000000 records in each file, and each file was processed within 10 minutes in Python.

The followup pairs of clinical providers are unique in this dataset. There is only one
record for each followup pair. To be included in the dataset a followup pair of clinical
providers must have had at least 11 clients in common in the calendar year of surveillance.
The `SourceProvider` and `TargetProvider` fields can be linked to the `Provider` fields in
the `Provider` and `Specialty` tables.

### Record Layout
|Field                   |Type      |Description                                                                                                                                    |
|------------------------|----------|-----------------------------------------------------------------------------------------------------------------------------------------------|
|`Source Provider`       |`CHAR(10)`|Initial clinical provider's unique identifier. Leading characters are meaningful, including zeros and spaces.                                  |
|`Target Provider`       |`CHAR(10)`|Followup clinical provider's unique identifier. Leading characters are meaningful, including zeros and spaces.                                 |
|`Followups Within Month`|`INT`     |Number of pairs of encounters between the initial clinical provider and the followup clinical provider that occurred within a month each other.|
|`Followups Within Day`  |`INT`     |Number of pairs of encounters between the initial clinical provider and the followup clinical provider that occurred on the same day.          |
|`Unique Clients`        |`INT`     |Number of clients seen in the pairs of encounters. Always less than or equal to the number of pairs of encounters.                             |

The three measure fields `Followups Within Month`, `Followups Within Day`, and
`Unique Clients` take values between 0 and less than 100000. A handful of records will
contain values exceeding 32000 and as such require the `INT` type, as `SMALLINT` is not
large enough.

### Quality Control

Regular Expression to find invalid records.

```Re
^(?!"[0-9]{10}","[0-9]{10}","[0-9]*","[0-9]*","[0-9]*"$)
```

### Files
There are seven to eight files for each calendar year, each containing no more than 7500000
records.Each team will be assigned a single calendar year of data.
* 2009 Calendar Year
  * Source `Data\pspp2009_30.csv`
  * target `Data\Clean\Followup2009-{1-7}.csv`
* 2010 Calendar Year
  * Source `Data\pspp2010_30.csv`
  * target `Data\Clean\Followup2010-{1-7}.csv`
* 2011 Calendar Year
  * Source `Data\pspp2011_30.csv`
  * target `Data\Clean\Followup2011-{1-8}.csv`
* 2012 Calendar Year
  * Source `Data\pspp2012_30.csv`
  * target `Data\Clean\Followup2012-{1-8}.csv`

Step through in blocks of 10000 appending to the file.

In [1]:
# Usual suspects
import pandas as pd
from csv import QUOTE_ALL

# Clean up, rename, remove columns
def cleanfollowups(source):
    source.replace("\"+", " ", regex = True)
    source.rename(
        columns = {
            "npi1": "Source Provider",
            "npi2": "Target Provider",
            "paircount": "Followups Within Month",
            "samedaycount": "Followups Within Day",
            "benecount": "Unique Clients"
        },
        inplace = True
    )
    target = source[
        [
            "Source Provider",
            "Target Provider",
            "Followups Within Month",
            "Followups Within Day",
            "Unique Clients"
        ]
    ]
    return target

# Batch read size
chunksize = 10**4
recordlimit = 75 * 10**5

### 2009 Calendar Year

In [2]:
# Batch read
with pd.read_csv("Data\pspp2009_30.csv", dtype = str, chunksize = chunksize) as followupreader:
    records = 0
    files = 0
    for followupsource in followupreader:
        followuptarget = cleanfollowups(followupsource)

        # Write with header
        if records % recordlimit == 0:
            files += 1
            followuptarget.to_csv(
                f"Data\Clean\Followup2009-{files}.csv",
                quoting = QUOTE_ALL,
                index = False
            )
        
        # Append without header
        else:
            followuptarget.to_csv(
                f"Data\Clean\Followup2009-{files}.csv",
                mode = "a",
                quoting = QUOTE_ALL,
                header = False,
                index = False
            )
        records += len(followuptarget.index)

        # Progress
        print(f"Records: {records} Files: {files}", end = "\r")

### 2010 Calendar Year

In [3]:
# Batch read
with pd.read_csv("Data\pspp2010_30.csv", dtype = str, chunksize = chunksize) as followupreader:
    records = 0
    files = 0
    for followupsource in followupreader:
        followuptarget = cleanfollowups(followupsource)

        # Write with header
        if records % recordlimit == 0:
            files += 1
            followuptarget.to_csv(
                f"Data\Clean\Followup2010-{files}.csv",
                quoting = QUOTE_ALL,
                index = False
            )
        
        # Append without header
        else:
            followuptarget.to_csv(
                f"Data\Clean\Followup2010-{files}.csv",
                mode = "a",
                quoting = QUOTE_ALL,
                header = False,
                index = False
            )
        records += len(followuptarget.index)

        # Progress
        print(f"Records: {records} Files: {files}", end = "\r")

### 2011 Calendar Year

In [4]:
# Batch read
with pd.read_csv("Data\pspp2011_30.csv", dtype = str, chunksize = chunksize) as followupreader:
    records = 0
    files = 0
    for followupsource in followupreader:
        followuptarget = cleanfollowups(followupsource)

        # Write with header
        if records % recordlimit == 0:
            files += 1
            followuptarget.to_csv(
                f"Data\Clean\Followup2011-{files}.csv",
                quoting = QUOTE_ALL,
                index = False
            )
        
        # Append without header
        else:
            followuptarget.to_csv(
                f"Data\Clean\Followup2011-{files}.csv",
                mode = "a",
                quoting = QUOTE_ALL,
                header = False,
                index = False
            )
        records += len(followuptarget.index)

        # Progress
        print(f"Records: {records} Files: {files}", end = "\r")

### 2012 Calendar Year

In [5]:
# Batch read
with pd.read_csv("Data\pspp2012_30.csv", dtype = str, chunksize = chunksize) as followupreader:
    records = 0
    files = 0
    for followupsource in followupreader:
        followuptarget = cleanfollowups(followupsource)

        # Write with header
        if records % recordlimit == 0:
            files += 1
            followuptarget.to_csv(
                f"Data\Clean\Followup2012-{files}.csv",
                quoting = QUOTE_ALL,
                index = False
            )
        
        # Append without header
        else:
            followuptarget.to_csv(
                f"Data\Clean\Followup2012-{files}.csv",
                mode = "a",
                quoting = QUOTE_ALL,
                header = False,
                index = False
            )
        records += len(followuptarget.index)

        # Progress
        print(f"Records: {records} Files: {files}", end = "\r")